In [4]:
import os
import sys
import tensorflow as tf
import tensorflow_io as tfio
import numpy as np

In [ ]:
# A function that will use the txt files in the data to create arrays of file paths  
#  basedir = ./data/
def txt_file_path_loader(file_path, base_dir = '/data/'):
    with open(file_path, 'r') as f:
        lines = f.readlines()
    file_paths = [base_dir + line.strip() for line in lines]
    return file_paths

In [ ]:
# converting stereo audio into mono for 1d convelution and training
def convert_to_mono(wav):
    return tf.reduce_mean(wav, axis = -1, keepdims= True)

In [ ]:
def resample_wav(wav, og_sample_rate, target_sample_rate):
        # Calculate the number of samples needed for 16kHz
        duration = tf.shape(wav)[0] / og_sample_rate
        new_sample_count = tf.cast(duration * target_sample_rate, tf.int32)
        
        # Resample using tf.image.resize with 1D signal
        resampled = tf.image.resize(
            tf.expand_dims(wav, -1), 
            [new_sample_count, 1], 
            method='bilinear'
        )
        
        return tf.squeeze(resampled)

In [ ]:
# A function to take in a file path and load and prepare an audio file
def prepare_wav(file_path, target_rate):
    # load audio file in
    audio_binary = tf.io.read_file(file_path)
    wav, sample_rate = tf.audio.decode_wav(audio_binary)

    # Resample to 16kHz if necessary
    if sample_rate != target_rate:
        wav = resample_wav(wav, sample_rate, target_rate)

    # Convert to mono by taking the first channel if stereo
    if wav.shape[-1] > 1 :
        wav = convert_to_mono(wav)

    return wav

    

In [ ]:
def convert_spectrogram(wav_tensor, frame_length, framestep):
    spectrogram = 

In [ ]:
# A function that will preprocess an audio file and make it ready for training
def preprocess_audio_file(audio_tensor):
    ...

In [ ]:
# A function that is going to take in a set of file paths and retrun the appriopriate X and Y object for that data
def prepare_dataset(X_file_paths, base_dir, batch_size, target_rate):
    ...

In [ ]:
#A function to create tensorflow data set for better GPU acceleration during training, validation and testing
def create_dataset(X_file_paths, base_dir, batch_size, target_rate ):
    ...